In [5]:
import sqlite3
import os
db_path = 'data/mimic-iv-clinical-database-demo-2.2/collated_data.db'

In [7]:

def get_table_headings(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Get the list of all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
  
    table_headings = {}
    
    # Iterate through the tables and get the headings
    for table in tables:
        table_name = table[0]
        # cursor.execute(f"PRAGMA table_info({table_name});")
        # columns = cursor.fetchall()
        # headings = [column[1] for column in columns]
        cursor = conn.execute('select * from ' + table_name)
        headings = [description[0] for description in cursor.description]
        table_headings[table_name] = headings
    
    # Close the connection
    conn.close()
    
    return table_headings

headings = get_table_headings(db_path)
print(headings)

{'admissions': ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'admit_provider_id', 'admission_location', 'discharge_location', 'insurance', 'language', 'marital_status', 'race', 'edregtime', 'edouttime', 'hospital_expire_flag'], 'diagnoses_icd': ['subject_id', 'hadm_id', 'seq_num', 'icd_code', 'icd_version'], 'drgcodes': ['subject_id', 'hadm_id', 'drg_type', 'drg_code', 'description', 'drg_severity', 'drg_mortality'], 'd_hcpcs': ['code', 'category', 'long_description', 'short_description'], 'd_icd_diagnoses': ['icd_code', 'icd_version', 'long_title'], 'd_icd_procedures': ['icd_code', 'icd_version', 'long_title'], 'd_labitems': ['itemid', 'label', 'fluid', 'category'], 'emar': ['subject_id', 'hadm_id', 'emar_id', 'emar_seq', 'poe_id', 'pharmacy_id', 'enter_provider_id', 'charttime', 'medication', 'event_txt', 'scheduletime', 'storetime'], 'emar_detail': ['subject_id', 'emar_id', 'emar_seq', 'parent_field_ordinal', 'administration_type', 'pharmacy_id',

In [9]:
# Number of tables
len(os.listdir('data/mimic-iv-clinical-database-demo-2.2/hosp'))

22

In [8]:
# Find out which tables contain the subject_id column
tables_with_subj_id_in = [a for a in headings if 'subject_id' in headings[a]]
tables_without_subj_id_in = [a for a in headings if 'subject_id' not in headings[a]]

In [28]:
print(tables_with_subj_id_in)
len(tables_with_subj_id_in)
print(tables_without_subj_id_in)

['admissions', 'diagnoses_icd', 'drgcodes', 'emar', 'emar_detail', 'hcpcsevents', 'labevents', 'microbiologyevents', 'omr', 'patients', 'pharmacy', 'poe', 'poe_detail', 'prescriptions', 'procedures_icd', 'services', 'transfers']
['d_hcpcs', 'd_icd_diagnoses', 'd_icd_procedures', 'd_labitems', 'provider']


In [34]:
conn=sqlite3.connect(db_path)
cursor=conn.cursor()

# Truncate table
# cursor.execute('SELECT subject_id, gender, anchor_age, dod FROM patients')
# selected_cols = cursor.fetchall()
# print(selected_cols)

# Get ID of first subject
cursor.execute('SELECT subject_id FROM patients LIMIT 1')
first_subject_id = cursor.fetchone()[0]

# Get selected columns
cursor.execute('SELECT gender, anchor_age, dod FROM patients WHERE subject_id = ?', (first_subject_id,))
gender, anchor_age, dod = cursor.fetchone()
print(gender, anchor_age, dod)

for t in tables_with_subj_id_in:
    cursor.execute(f'SELECT * FROM {t} WHERE subject_id = ?', (first_subject_id,))

    output_dir = 'data/mimic-iv-clinical-database-demo-2.2/subject-data'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Write output
    with open(os.path.join(output_dir, f'{first_subject_id}.txt'), 'a') as f:
        for row in cursor.fetchall():
            f.write(str(row) + '\n')

    # print(t)
    # print(cursor.fetchall())

conn.close()

F 21 None


In [16]:
# Iterate over patients and generate output files
conn = sqlite3.connect(db_path)
cursor=conn.cursor()
cursor2=conn.cursor()
cursor.execute('SELECT subject_id FROM patients')

for patient in cursor:
    for t in tables_with_subj_id_in:
        cursor2.execute(f'SELECT * FROM {t} WHERE subject_id = ?', (patient[0],))

        output_dir = 'data/mimic-iv-clinical-database-demo-2.2/subject-data'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Write output
        with open(os.path.join(output_dir, f'{patient[0]}.txt'), 'a') as f:
            for row in cursor2.fetchall():
                f.write(str(row) + '\n')

conn.close()